## File Encryption/Decryption using AES and RSA

This program utilizes the python hazmat cryptography library to encrypt and decrypt user files. The file will first be padded using PKCS7 to make sure the input to our encryption algorithm is the appropriate block size. CBC mode (AES) will then be used to encrypt a file of the user's choice. The encryption of a file is further enhanced with RSA encryption on the AES encryption key, and the program will also write the rsa public and private key to files. The output of the program will be a JSON object that contains the ciphertext, iv, RSA encrypted encryption key, and the file extension. The JSON object can then be read and then decrypted using the RSA private key to decrypt the encryption key and then using the encryption key to decrypt the ciphertext.

In [1]:
# import the necessary libraries needed
import os as os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding as asymPadding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import json

In [2]:
# constants used for our encryption algorithm
IV_LENGTH = 16
KEY_LENGTH = 32
BLOCK_SIZE = 128

In [3]:
def myEncrypt(message):
    """
    Encrypts the message using CBC mode (AES).
    
    Arguments:
    message -- stream of bytes the user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    """
    
    # generate our key and iv
    backend = default_backend()
    iv = os.urandom(IV_LENGTH)
    key = os.urandom(KEY_LENGTH)
    
    # pad to approprate block size
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    paddedMessage = padder.update(message)
    paddedMessage += padder.finalize()
    
    # encrypt using CBC mode (AES)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    encryptor = cipher.encryptor()
    ct = encryptor.update(paddedMessage) + encryptor.finalize()
    
    return ct, iv, key

In [5]:
def myFileEncrypt(filepath):
    """
    Takes the file from the filepath specified by the user and encrypts the file. 
    
    Arguments:
    filepath -- Filepath for the file user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    fileType -- the file extension
    """
    # read the file and convert it to bytes
    file = open(filepath, "rb")
    byte = bytes(file.read())
    file.close()
    
    # encrypt the file
    cipher, iv, key = myEncrypt(byte)
    fileType = "PNG"
    return cipher, iv, key, fileType

In [7]:
def myRSAEncrypt(filepath):
    """
    Takes the file from the filepath specified by the user and encrypts the file and also the encryption key. 
    
    Arguments:
    filepath -- Filepath for the file user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    RSAencryptedKey -- the RSA encrypted encryption key needed for decryption
    fileType -- the file extension
    """
    # Generate RSA private key and public key
    private_key = rsa.generate_private_key(
        public_exponent = 65537,
        key_size = 2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    # Write out RSA private key and public key to file using serialization
    RSAprivateKey = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    RSApublicKey = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo,
    )
    rsaFile = open("RSAprivateKey.pem", "wb")
    rsaFile.write(RSAprivateKey)
    rsaFile.close()

    rsaFile = open("RSApublicKey.pem", "wb")
    rsaFile.write(RSApublicKey)
    rsaFile.close()

    # Encrypt our file of choice using myFileEncrypt()
    cipher, iv, encryptionKey, fileType = myFileEncrypt(filepath)

    # Encrypt the encryption key using the RSA public key with OAEP
    RSAencryptedCipher = public_key.encrypt(
        encryptionKey,
        asymPadding.OAEP(
            mgf=asymPadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return cipher, iv, RSAencryptedCipher, fileType

In [8]:
def myDecrypt(message, key, iv):
    """
    Decrypts the message using CBC mode (AES).
    
    Arguments:
    message -- stream of bytes the user wants to encrypt
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    
    Return:
    unpaddedMessage -- the original message
    """
    
    # decrypt the file
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    decryptedMessage = decryptor.update(message) + decryptor.finalize()
    
    # unpad the file
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    unpaddedMessage = unpadder.update(decryptedMessage)
    
    return unpaddedMessage

In [9]:
def myFileDecrypt(filepath, iv, key, fileType):
    """
    Takes the file from the filepath specified by the user and encrypts the file. 
    
    Arguments:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    fileType -- the file extension
    """    
    file = open(filepath, "rb")
    byte = bytes(file.read())
    decryptedFile = myDecrypt(byte, key, iv)
    image = Image.open(io.BytesIO(decryptedFile))
    image.show()

In [4]:
# test case for a string
message = "A Secret Message for Mehrdad Aliasgari the great"
byteMessage = message.encode('utf-8')
print(type(byteMessage))
print(byteMessage)
testMsg, testIV, testKey = myEncrypt(byteMessage)

<class 'bytes'>
b'A Secret Message for Mehrdad Aliasgari the great'


In [8]:
# Encrypt a file of the user's choice
cipher, iv, RSAencryptedCipher, fileType = myRSAEncrypt("test.png")

files = {
    "cipher" : str(cipher),
    "iv" : str(iv),
    "key": str(RSAencryptedCipher),
    "ext"  : fileType   
}

fileName = open("encrypt.json", "w")
jsonObject = json.dump(files,fileName)
fileName.close()

In [11]:
# Decrypt the file of the user's choice
fileName = open("encrypt.json", "r")
jsonObject = json.load(fileName)
cipher = jsonObject["cipher"]
iv = jsonObject["iv"]
RSAencryptedKey = jsonObject["key"]
fileType = jsonObject["ext"]
fileName.close()


In [11]:
##myFileDecrypt("out.bytes",iv, key, "PNG")